In [ ]:
%load_ext autoreload
%autoreload 2

import sys
if '..' not in sys.path:
    sys.path.append('..')

from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [ ]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

def noise(seq):
    toks = tokenizer(seq, max_length=512, truncation=True)['input_ids'][1:-1]
    traj = [tokenizer.decode(toks)]
    N = len(toks)
    
    while toks:
        toks = toks[:len(toks)//2]
        traj.append(tokenizer.decode(toks))
        
    return traj[::-1], 0, N

In [ ]:
import os
import json
from tqdm import tqdm

lengths = []

with open('../data/imdb/imdb_train_3.jsonl', 'w') as train_f, open('../data/imdb/imdb_dev_3.jsonl', 'w') as dev_f:
    for i, file in tqdm(enumerate(os.listdir('../data/imdb/raw/train/neg'))):
        df = train_f if i < 10000 else dev_f
        with open(f'../data/imdb/raw/train/neg/{file}', 'r') as f:
            seq = f.read()
            *traj, N = noise(seq)
            json.dump(traj, df)
            df.write('\n')
            lengths.append(N)
            
    for i, file in tqdm(enumerate(os.listdir('../data/imdb/raw/train/pos'))):
        df = train_f if i < 10000 else dev_f
        with open(f'../data/imdb/raw/train/pos/{file}', 'r') as f:
            seq = f.read()
            *traj, N = noise(seq)
            json.dump(traj, df)
            df.write('\n')
            lengths.append(N)

In [ ]:
import json
import torch
import pickle
from transformers import BertTokenizer
from constants import *
from tqdm import tqdm

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

with open('../data/imdb/imdb_train_3.jsonl', 'r') as f:
    for i, line in tqdm(enumerate(f.readlines())):
        if not line: continue
        traj, _ = json.loads(line)
        ids = torch.tensor(traj)
        
        traj_op_tgts = []
        traj_tok_tgts = []
        traj_idx_tgts = []
        
        for j, seq in enumerate(ids[1:], start=1):
            N = len(ids[j-1]) - 2
            M = len(seq) - 2 - N
            
            op_tgts = [INS_ID] + [CPY_ID for _ in range(N)] + [INS_ID for _ in range(M)] + [EOS_ID]
            tok_tgts = [BOS_TOKEN_ID] + [PAD_TOKEN_ID for _ in range(N)] + [tok for tok in seq[1+N:1+N+M]] + [PAD_TOKEN_ID]
            idx_tgts = [0]  + [i for i in range(1, N+1)] + [0 for _ in range(M)] + [0]
           
            # pad out to 512 
            op_tgts += [PAD_ID for _ in range(512-len(seq))]
            tok_tgts += [PAD_TOKEN_ID for _ in range(512-len(seq))]
            idx_tgts += [0 for _ in range(512-len(seq))]
            
            traj_op_tgts.append(op_tgts)
            traj_tok_tgts.append(tok_tgts)
            traj_idx_tgts.append(idx_tgts)
            
        traj_op_tgts = torch.tensor(traj_op_tgts)
        traj_tok_tgts = torch.tensor(traj_tok_tgts)
        traj_idx_tgts = torch.tensor(traj_idx_tgts)
        
        with open(f'../data/imdb/cache_3/sup-imdb-3_{i}.zst', 'wb') as f:
            pickle.dump((traj_op_tgts, traj_tok_tgts, traj_idx_tgts), f)

In [ ]:
from data import elaborate
import torch.nn.functional as F

with open('../data/imdb/imdb_train_2.jsonl', 'r') as f:
    for i, line in tqdm(enumerate(f.readlines())):
        if not line: continue
        traj, _ = json.loads(line)
        ids = torch.tensor(traj)
        
        traj_op_tgts = []
        traj_tok_tgts = []
        traj_idx_tgts = []
        
        for j, seq in enumerate(ids[1:], start=1):
            N = len(ids[j-1]) 
            assert N == len(seq)
            
            op_tgts = [PAD_ID for _ in range(512)]
            tok_tgts = [PAD_TOKEN_ID for _ in range(512)]
            idx_tgts = [0 for _ in range(512)]
            
            op_tgts[0] = INS_ID
            tok_tgts[0] = BOS_TOKEN_ID
            
            op_tgts[N-1] = EOS_ID
            
            for k, (a, b) in enumerate(zip(ids[j-1][1:-1], seq[1:-1]), start=1):
                if a == b:
                    op_tgts[k] = CPY_ID
                    idx_tgts[k] = k
                
                else:
                    assert a.item() == 103
                    op_tgts[k] = SUB_ID
                    tok_tgts[k] = b.item()
                    idx_tgts[k] = k
                    
            traj_op_tgts.append(op_tgts)
            traj_tok_tgts.append(tok_tgts)
            traj_idx_tgts.append(idx_tgts)
            
        traj_op_tgts = torch.tensor(traj_op_tgts)
        traj_tok_tgts = torch.tensor(traj_tok_tgts)
        traj_idx_tgts = torch.tensor(traj_idx_tgts)
        
        with open(f'../data/imdb/cache_2/sup-imdb-2_{i}.zst', 'wb') as f:
            pickle.dump((traj_op_tgts, traj_tok_tgts, traj_idx_tgts), f)

In [ ]:
import torch.nn.functional as F

traj_op_tgts = F.one_hot(traj_op_tgts, 5)
traj_tok_tgts = F.one_hot(traj_tok_tgts, VOCAB_SIZE)
traj_idx_tgts = F.one_hot(traj_idx_tgts, 512)

In [ ]:
from data import elaborate

elaborate((traj_op_tgts, traj_tok_tgts, traj_idx_tgts))

## inspection

sup-imdb-1.3: fixed loss composition and achieves best eval out of this batch of experiments, but the index head plateaus and op loss isn't great

sup-imbd-1.4: added scaling to index and op heads, which resulted in much lower loss, but eval is significaly worse

what gives?

In [ ]:
import torch
from model import Evolver
from data import get_input_ids

model_13 = Evolver(d_model=512, max_len=512)
model_13.load_state_dict(torch.load('../checkpoints/sup-imdb-1.3.pt', map_location='cpu')['model'])
model_13.eval()

model_14 = Evolver(d_model=512, max_len=512)
model_14.load_state_dict(torch.load('../checkpoints/sup-imdb-1.4.pt', map_location='cpu')['model'])
model_14.eval()

In [ ]:
inputs = ['very unnecessary movie with']
traj_input_ids = get_input_ids(inputs, 512, tokenizer)

tokenizer('very unnecessary movie with characters that are acting so')['input_ids']

In [ ]:
import torch.nn.functional as F
from constants import *
from data import get_edit_tgts

edit_tgts = get_edit_tgts(
    [
        (INS_ID, BOS_TOKEN_ID, 0),
        (CPY_ID, PAD_TOKEN_ID, 1),
        (CPY_ID, PAD_TOKEN_ID, 2),
        (CPY_ID, PAD_TOKEN_ID, 3),
        (CPY_ID, PAD_TOKEN_ID, 4),
        (INS_ID, 3494, 0),
        (INS_ID, 2008, 0),
        (INS_ID, 2024, 0),
        (INS_ID, 3772, 0),
        (INS_ID, 2061, 0),
    ],
    max_len=512
)

op_tgts = F.one_hot(torch.tensor(edit_tgts[0]), 5).unsqueeze(0)
tok_tgts = F.one_hot(torch.tensor(edit_tgts[1]), VOCAB_SIZE).unsqueeze(0)
idx_tgts = F.one_hot(torch.tensor(edit_tgts[2]), 512).unsqueeze(0)

from data import elaborate
elaborate((op_tgts, tok_tgts, idx_tgts))

In [ ]:
model_14.train()
src, src_pad_mask = model_14.get_src(traj_input_ids)
probs_14, tgt, _, _ = model_14.forward(traj_input_ids, src, (op_tgts, tok_tgts, idx_tgts), src_pad_mask, None, None, None)

model_13.train()
src, src_pad_mask = model_13.get_src(traj_input_ids)
probs_13, tgt, _, _ = model_13.forward(traj_input_ids, src, (op_tgts, tok_tgts, idx_tgts), src_pad_mask, None, None, None)

# from tqdm import tqdm

# a, b, c = [], [], []
# memory = cache = None
# src, src_pad_mask = model_14.get_src(traj_input_ids)

# for i in tqdm(range(1, 6)):
#     edit_tgts = tuple(map(
#         lambda x: x[:, :i],
#         (op_tgts, tok_tgts, idx_tgts)
#     ))
    
#     probs, tgt, memory, cache = model_14.forward(traj_input_ids, src, edit_tgts, src_pad_mask, None, memory, cache)
#     op_probs, tok_probs, idx_probs = tuple(map(lambda x: x[:, -1].squeeze().detach(), probs))
    
#     print('best token', torch.argmax(tok_probs))
#     print('best index', torch.argmax(idx_probs))
       
#     a.append(torch.exp(op_probs).numpy())
#     b.append(torch.exp(tok_probs).numpy())
#     c.append(torch.exp(idx_probs).numpy())

In [ ]:
# torch.set_printoptions(threshold=float('inf'))

print((probs_14[0][:, :10]))
print((probs_13[0][:, :10]))

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(5, 10))

im1 = ax1.imshow(np.array(a).T, aspect='auto', cmap='plasma')
plt.colorbar(im1, ax=ax1)

im2 = ax2.imshow(np.array(b).T, aspect='auto', cmap='plasma')
plt.colorbar(im2, ax=ax2)

im3 = ax3.imshow(np.array(c).T, aspect='auto', cmap='plasma')
plt.colorbar(im3, ax=ax3)

plt.tight_layout()
plt.show()

In [ ]:
from data import TrajectoryDataset

dataset = TrajectoryDataset.from_disk(
    '../data/imdb/imdb_dev_1.jsonl',
    max_len=512,
    tokenizer=tokenizer
)

In [ ]:
from torch.utils.data import DataLoader
from train import evolver_elbo
from data import collate_unsupervised

loader = DataLoader(dataset, batch_size=1, collate_fn=collate_unsupervised)

loss = evolver_elbo(model_13, loader, 1, 'cpu')